<a href="https://colab.research.google.com/github/rsemihkoca/YouDo-ds-bc/blob/main/YouDo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import numpy as np


class UserBased:
    mu: np.ndarray
    sim: np.ndarray

    def __init__(self, zero_mean: bool = True, beta: int = 1, idf: bool = False, verbosity: int = 0):

        self.zero_mean = zero_mean
        self.beta = beta
        self.idf = idf
        self.verbosity = verbosity

    def fit(self, r: np.ndarray):
        m, n = r.shape
        if self.zero_mean:
            self.mu = np.nanmean(r, axis=1)
        else:
            self.mu = np.zeros(m)

        self.sim = np.zeros((m, m))

        if self.idf:
            idf = np.log(1 + m / (~np.isnan(r)).sum(axis=0))
        else:
            idf = np.ones(n)

        if self.verbosity > 0:
            print(idf)

        for i in range(m):
            for j in range(m):
                mask = ~np.isnan(r[i, :]) & ~np.isnan(r[j, :])

                si = r[i, mask] - self.mu[i]
                sj = r[j, mask] - self.mu[j]

                self.sim[i][j] = (si * sj * idf[mask]).sum() / (
                        np.sqrt((idf[mask] * (si ** 2)).sum()) * np.sqrt((idf[mask] * (sj ** 2)).sum()))

                total_intersection = mask.sum()

                self.sim[i][j] *= min(total_intersection, self.beta) / self.beta

        return self.sim

    def predict_single_rate(self, r: np.array, u: int, j: int, top_k: int = 3) -> float:
        _, n = r.shape

        users_rated_j = np.nonzero(~np.isnan(r[:, j]))[0]

        topk_users = users_rated_j[self.sim[u, users_rated_j].argsort()[::-1][:top_k]]

        mean_centered_topk_user_rate = r[topk_users, j] - self.mu[topk_users]
        Rvj=r[topk_users, j] # topk k similar userin j itemine vermiş oldukları rating vectoru
        # w = self.sim[u, topk_users]
        Mv=self.mu[topk_users]
        # return np.dot(mean_centered_topk_user_rate, w) / np.abs(w).sum() + self.mu[u]
        Mu=self.mu[u]

        return Rvj,Mv,Mu # (rvj-mv) ve mv

   


# r = np.array([[7, 6, 7, 4, 5, 4,6],
#               [6, 7, np.nan, 4, 3, 4,7],
#               [np.nan, 3, 3, 1, 1, np.nan,3],
#               [1, 2, 3, 3, 3, 4,2],
#               [1, np.nan, 1, 2, 3, 3,3],
#               [1, 5, 1, 2, 3, 3,5]])

# u=4 # 5. user 
# j=1 # 2. item nan values
# UserBased_model = UserBased(zero_mean=False, beta=1, idf=False)

# sim = UserBased_model.fit(r)

# r_userbased=UserBased_model.predict_single_rate(r,u,j, top_k=1)
# r_userbased

# j,u=u,j
# u
# u=1  2. item 
# j=4  5. user nan values
# ItemBased_model = UserBased(zero_mean=False, beta=1, idf=False)

# sim = ItemBased_model.fit(r.T)

# r_itembased=ItemBased_model.predict_single_rate(r.T,u,j, top_k=1)


# r_itembased
        
import pandas as pd
r = pd.DataFrame([[7, 6, 7, 4, 5, 4,6],
              [6, 7, np.nan, 4, 3, 4,7],
              [np.nan, 3, 3, 1, 1, np.nan,3],
              [1, 2, 3, 3, 3, 4,2],
              [1, np.nan, 1, 2, 3, 3,3],
              [1, 5, 1, 2, 3, 3,5]])
r.index =['User_0','User_1', 'User_2', 'User_3', 'User_4','User_4_cpy']
r.columns =['Item_0','Item_1', 'Item_2', 'Item_3', 'Item_4', 'Item_5','Item_1_cpy']
r,r.T
# r_pred=np.full((r.shape),np.nan)
# r_pred    

(            Item_0  Item_1  Item_2  Item_3  Item_4  Item_5  Item_1_cpy
 User_0         7.0     6.0     7.0       4       5     4.0           6
 User_1         6.0     7.0     NaN       4       3     4.0           7
 User_2         NaN     3.0     3.0       1       1     NaN           3
 User_3         1.0     2.0     3.0       3       3     4.0           2
 User_4         1.0     NaN     1.0       2       3     3.0           3
 User_4_cpy     1.0     5.0     1.0       2       3     3.0           5,
             User_0  User_1  User_2  User_3  User_4  User_4_cpy
 Item_0         7.0     6.0     NaN     1.0     1.0         1.0
 Item_1         6.0     7.0     3.0     2.0     NaN         5.0
 Item_2         7.0     NaN     3.0     3.0     1.0         1.0
 Item_3         4.0     4.0     1.0     3.0     2.0         2.0
 Item_4         5.0     3.0     1.0     3.0     3.0         3.0
 Item_5         4.0     4.0     NaN     4.0     3.0         3.0
 Item_1_cpy     6.0     7.0     3.0     2.0    

In [227]:
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
names=['user_id', 'item_id', 'rating', 'timestamp'])
r = df.pivot(index='user_id', columns='item_id', values='rating')
print(r[r.isnull().all(axis=0)],r.shape)
r = df.pivot(index='user_id', columns='item_id', values='rating').values


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Empty DataFrame
Columns: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, ...]
Index: []

[0 rows x 1682 columns] (943, 1682)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [220]:
pd.DataFrame(r).head(5)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [228]:
import numpy as np
# r_init = np.array([[7, 6, 7, 4, 5, 4,6],
#               [6, 7, np.nan, 4, 3, 4,7],
#               [np.nan, 3, 3, 1, 1, np.nan,3],
#               [1, 2, 3, 3, 3, 4,2],
#               [1, np.nan, 1, 2, 3, 3,3],
#               [1, 5, 1, 2, 3, 3,5]])
r=r.copy()
#r=r_init.copy()
ItemBased_model = UserBased(zero_mean, beta, idf)
ItemBased_model.fit(r.T)

UserBased_model = UserBased(zero_mean, beta, idf)
UserBased_model.fit(r)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


array([[ 1.,  0., ..., -0.,  0.],
       [ 0.,  1., ...,  0.,  1.],
       ...,
       [-0.,  0., ...,  1.,  0.],
       [ 0.,  1., ...,  0.,  1.]])

In [233]:

# def Item_based_model(r,u,j,top_k):

#   Rut,Mj,Mu=ItemBased_model.predict_single_rate(r,u,j, top_k)
#   return Rut,Mj,Mu

# def User_based_model(r,u,j,top_k):

#   Rvj,Mv,Mu=UserBased_model.predict_single_rate(r,u,j, top_k)
#   return Rvj,Mv,Mu


def Combined_SGD(r,ItemBased_model,UserBased_model,top_k,beta,zero_mean,idf,lamb,alpha,max_iter):
  users,items=r.shape
  w_users=np.random.rand(users,top_k)
  w_items=np.random.rand(top_k,items)
  grad_users=np.zeros(top_k)
  grad_items=np.zeros(top_k)

  ItemBased_model = ItemBased_model
  UserBased_model = UserBased_model

  r_pred=np.full((r.shape),np.nan)

  for iter in range(max_iter):
    loss=0
    Cum_loss=0
    for u in range(users): 
      for j in range(items): 
        if np.isnan(r[u][j]): # zaten bilmediğim değeri tahmin edemem NANları nasıl tahmin ederim
          continue
        else:
          
          Rvj,Mv,Mu=UserBased_model.predict_single_rate(r,u,j, top_k)
          Rut,Mj,Mt=ItemBased_model.predict_single_rate(r.T,j,u, top_k)

          r_pred[u][j]= Mu + np.dot(w_users[u,:],Rvj-Mv) + np.dot(w_items[:,j],Rut-Mj)

          loss+= (r[u][j] - r_pred[u][j])**2 #train test splitte ayır bunları

          grad_users = -2*np.dot((r[u][j] - r_pred[u][j]),(Rvj-Mv))
          grad_items = -2*np.dot((r[u][j] - r_pred[u][j]),(Rut-Mj))

          w_users_prev=w_users.copy()
          w_items_prev=w_items.copy()

          w_users[u,:] = w_users[u,:] - alpha * grad_users
          w_items[:,j] = w_items[:,j] - alpha * grad_items
          
          #Cum_loss+= np.nansum((r-r_pred)**2)
          

      if(j%10 == 0):
          print(f"iteration: ({iter}) , gradient: {np.linalg.norm(grad_users)+np.linalg.norm(grad_items):.3f}, loss={loss:.3f}, Total error: {Cum_loss:.3f}")
                
      # if np.linalg.norm(w_users-w_users_prev)+np.linalg.norm(w_items-w_items_prev) < 0.000002:
      #     print(f"I do early stoping at iteration {iter}")
      #     break
  return r_pred,w_users,w_items

#users,items=r.shape
# w_users=np.random.rand(users,top_k)
# w_items=np.random.rand(top_k,items)
# grad=np.zeros(top_k)
top_k=1
alpha=(3)*10**-2
max_iter=20
beta=1
zero_mean=True
lamb=1
idf=False

r_pred,w_users,w_items=Combined_SGD(r,ItemBased_model,UserBased_model,top_k,beta,zero_mean,idf,lamb,alpha,max_iter)
np.set_printoptions()

In [238]:
np.nanmean((r-r_pred)**2)**0.5

0.20403047063932178

In [119]:
def topla(a,b):
  return a+b
def main():
  a=5
  b=6
  return topla(a,b)
main()

11

In [102]:
pd.DataFrame(w_users)

,0,1,2,3
0,0.336198,0.218681,-0.338324,-0.061333
1,0.202620,-0.098350,-0.221889,-0.203923
2,0.233252,-0.005706,-0.920284,0.235778
3,0.804346,-0.033304,0.350887,0.435210
4,-0.109812,0.382028,0.208525,0.060849
5,0.295427,-0.093575,-0.064323,0.106154


In [ ]:
np.array([[6.99701826, 6.01365855, 6.99771103, 3.99912263, 5.00242315, 3.99538269, 5.9838271 ],
       [5.9989501 , 6.98556526,        np.nan, 4.00205566, 3.00083937, 3.99778344, 7.01523193],
       [       np.nan, 2.99561491, 3.00100138, 0.99857788, 1.00199939, np.nan, 3.00501033],
       [1.00246134, 1.9983931 , 3.00104609, 2.9972323 , 3.00156812, 4.00365659, 2.00315068],
       [0.99010626,        np.nan, 1.01786441, 1.97618874, 3.04394068, 2.97602526, 2.99476216],
       [1.00198988, 5.00946122, 0.97679499, 2.02849337, 2.96021514, 3.01236376, 4.98515699]])

array([[6.99701826, 6.01365855, 6.99771103, 3.99912263, 5.00242315,
        3.99538269, 5.9838271 ],
       [5.9989501 , 6.98556526,        nan, 4.00205566, 3.00083937,
        3.99778344, 7.01523193],
       [       nan, 2.99561491, 3.00100138, 0.99857788, 1.00199939,
               nan, 3.00501033],
       [1.00246134, 1.9983931 , 3.00104609, 2.9972323 , 3.00156812,
        4.00365659, 2.00315068],
       [0.99010626,        nan, 1.01786441, 1.97618874, 3.04394068,
        2.97602526, 2.99476216],
       [1.00198988, 5.00946122, 0.97679499, 2.02849337, 2.96021514,
        3.01236376, 4.98515699]])

# R init için optimal model

In [217]:
import numpy as np
r = np.array([[7, 6, 7, 4, 5, 4,6],
              [6, 7, np.nan, 4, 3, 4,7],
              [np.nan, 3, 3, 1, 1, np.nan,3],
              [1, 2, 3, 3, 3, 4,2],
              [1, np.nan, 1, 2, 3, 3,3],
              [1, 5, 1, 2, 3, 3,5]])


def Item_based_model(r,u,j,top_k):

  Rut,Mj,Mu=ItemBased_model.predict_single_rate(r,u,j, top_k)
  return Rut,Mj,Mu

def User_based_model(r,u,j,top_k):

  Rvj,Mv,Mu=UserBased_model.predict_single_rate(r,u,j, top_k)
  return Rvj,Mv,Mu


def Combined_SGD(r,top_k,beta,zero_mean,idf,lamb,alpha,max_iter):
  users,items=r.shape
  w_users=np.random.rand(users,top_k)
  w_items=np.random.rand(top_k,items)
  grad_users=np.zeros(top_k)
  grad_items=np.zeros(top_k)


  

  r_pred=np.full((r.shape),np.nan)

  for iter in range(max_iter):
    loss=0
    Cum_loss=0

    for u in range(users):
      for j in range(items):
        if np.isnan(r[u][j]): # zaten bilmediğim değeri tahmin edemem NANları nasıl tahmin ederim
          continue
        else:
          ItemBased_model = UserBased(zero_mean=zero_mean, beta=beta, idf=idf)
          ItemBased_model.fit(r.T)

          UserBased_model = UserBased(zero_mean=zero_mean, beta=beta, idf=idf)
          UserBased_model.fit(r)
          
          Rvj,Mv,Mu=UserBased_model.predict_single_rate(r,u,j, top_k)
          Rut,Mj,Mu=ItemBased_model.predict_single_rate(r.T,j,u, top_k)

          r_pred[u][j]= Mu + np.dot(w_users[u,:],Rvj-Mv) + np.dot(w_items[:,j],Rut-Mj)

          loss+= (r[u][j] - r_pred[u][j])**2 #train test splitte ayır bunları

          grad_users = -2*np.dot((r[u][j] - r_pred[u][j]),(Rvj-Mv))
          grad_items = -2*np.dot((r[u][j] - r_pred[u][j]),(Rut-Mj))

          w_users_prev=w_users.copy()
          w_items_prev=w_items.copy()

          w_users[u,:] = w_users[u,:] - alpha * grad_users
          w_items[:,j] = w_items[:,j] - alpha * grad_items
          
          Cum_loss+= np.nansum((r-r_pred)**2)
          

    if(iter%10 == 0):
        print(f"iteration: ({iter}) , gradient: {np.linalg.norm(grad_users)+np.linalg.norm(grad_items):.3f}, loss={loss:.3f}, Total error: {Cum_loss:.3f}")
        
    if np.linalg.norm(w_users-w_users_prev)+np.linalg.norm(w_items-w_items_prev) < 0.000002:
        print(f"I do early stoping at iteration {iter}")
        break
  return r_pred,w_users,w_items

#users,items=r.shape
# w_users=np.random.rand(users,top_k)
# w_items=np.random.rand(top_k,items)
# grad=np.zeros(top_k)
r=r
top_k=2
alpha=(3)*10**-2
max_iter=100
beta=1
zero_mean=True
lamb=1
idf=False
r_pred,w_users,w_items=Combined_SGD(r,top_k,beta,zero_mean,idf,lamb,alpha,max_iter)
np.set_printoptions(precision=0, suppress=True)
print(r_pred)

iteration: (0) , gradient: 7.887, loss=214.037, Total error: 5978.259
iteration: (10) , gradient: 3.669, loss=2.014, Total error: 82.754
iteration: (20) , gradient: 3.319, loss=0.741, Total error: 29.168
iteration: (30) , gradient: 2.661, loss=0.401, Total error: 15.713
iteration: (40) , gradient: 1.982, loss=0.216, Total error: 8.456
iteration: (50) , gradient: 1.430, loss=0.118, Total error: 4.599
iteration: (60) , gradient: 1.026, loss=0.068, Total error: 2.660
iteration: (70) , gradient: 0.744, loss=0.043, Total error: 1.677
iteration: (80) , gradient: 0.552, loss=0.030, Total error: 1.154
iteration: (90) , gradient: 0.421, loss=0.022, Total error: 0.857
[[ 7.  6. ...  4.  6.]
 [ 6.  7. ...  4.  7.]
 ...
 [ 1. nan ...  3.  3.]
 [ 1.  5. ...  3.  5.]]


In [ ]:
grad=np.zeros(top_k)
grad

array([0., 0., 0.])

In [ ]:
users,items=r.shape
top_k=3
w_users=np.random.rand(1,top_k)
loss=0
grad=np.zeros(top_k)
for u in range(users):
  for j in range(items):
    if np.isnan(r[u][j]):
      continue
    Rvj,Mv,Mu=UserBased_model.predict_single_rate(r,u,j, top_k=3) # bunun icine itemi de gom tek seferde hallet
    loss+= (r[u][j] - Mu + np.dot(w_users,Rvj-Mv))**2
print(loss)

[334.73485311]


In [ ]:
def User_Item_Combined(x, y): # -> np.ndarray:
    theta=5
    beta = np.random.random(2)
    alpha=1.2*10**-2

    for i in range(1000000):
        y_pred= beta[0] + beta[1] * x    
       
        # Update Functions:
        g_b0 = -2 * ((y - y_pred).mean()) * np.power(np.e,(-1*np.power(((y - y_pred).mean()),2)/theta)) 
        g_b1 = -2 * ((x * (y - y_pred)).mean()) * np.power(np.e,(-1*np.power(((y - y_pred).mean()),2)/theta)) 

        beta_prev = np.copy(beta)


        beta[0] = beta[0] - alpha * g_b0
        beta[1] = beta[1] - alpha * g_b1
        
        if(i%1000 == 0):
            print(f"iteration: ({i}) beta: {beta}, gradient: {g_b0} {g_b1}")
            
        if np.linalg.norm(beta - beta_prev) < 0.00000001:
            print(f"I do early stoping at iteration {i}")
            break


    return beta

In [ ]:
beta=User_Item_Combined(X, y)
beta

In [ ]:
r.T

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
names=['user_id', 'item_id', 'rating', 'timestamp'])
r = df.pivot(index='user_id', columns='item_id', values='rating').values

In [ ]:
r

# Model 1

Part *1*:

In [ ]:
Casting yapmalı mıyım 4.7 to 5 gibi
mu iki kere ekleme bir kere eklenecek

In [ ]:
r = np.array([[7, 6, 7, 4, 5, 4],
              [6, 7, np.nan, 4, 3, 4],
              [np.nan, 3, 3, 1, 1, np.nan],
              [1, 2, 3, 3, 3, 4],
              [1, np.nan, 1, 2, 3, 3]])

In [ ]:
r

In [ ]:
m, n = r.shape
mu = np.nanmean(r, axis=1)
beta=3


sim = np.zeros((m, m))
sim

In [ ]:


idf = np.log(1 + m / (~np.isnan(r)).sum(axis=0))
idf

In [ ]:



for i in range(m):
    for j in range(m):
        mask = ~np.isnan(r[i, :]) & ~np.isnan(r[j, :])

        si = r[i, mask] - mu[i]
        sj = r[j, mask] - mu[j]

        sim[i][j] = (si * sj * idf[mask]).sum() / (
                np.sqrt((idf[mask] * (si ** 2)).sum()) * np.sqrt((idf[mask] * (sj ** 2)).sum()))

        total_intersection = mask.sum()

        sim[i][j] *= min(total_intersection, beta) / beta

sim

In [ ]:
r

In [ ]:
j= 3
i= 
mask = ~np.isnan(r[i, :]) & ~np.isnan(r[j, :])


In [ ]:
topk_users = users_rated_j[self.sim[u, users_rated_j].argsort()[::-1][:top_k]]

mean_centered_topk_user_rate = r[topk_users, j] - self.mu[topk_users]

w = self.sim[u, topk_users]

result= np.dot(mean_centered_topk_user_rate, w) / np.abs(w).sum() + self.mu[u]

In [ ]:
r_row,r_col=r.shape

In [ ]:
for u in range(r_row): # user sayısı kadar don
  for j in range(r_col): # item sayısı kadar don
  